<a href="https://colab.research.google.com/github/LordLean/Extracting-Green-Bonds-Use-of-Proceeds/blob/main/QA_FinBert_Finetuned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 4.1 MB/s 
     |████████████████████████████████| 101 kB 12.2 MB/s 
     |████████████████████████████████| 596 kB 67.3 MB/s 
     |████████████████████████████████| 6.6 MB 60.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 4.3 MB/s 
     |████████████████████████████████| 115 kB 79.8 MB/s 
     |████████████████████████████████| 212 kB 78.7 MB/s 
     |████████████████████████████████| 141 kB 13.3 MB/s 
     |████████████████████████████████| 127 kB 86.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully u

In [ ]:
from datasets import load_dataset

dataset = load_dataset("squad")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Dataset squad downloaded and prepared to /root/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
model_checkpoint = "yiyanghkust/finbert-pretrain"

In [ ]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [ ]:
max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.

In [ ]:
pad_on_right = tokenizer.padding_side == "right"

In [ ]:
def prepare_train_features(examples):

    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. 
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings give a map from token to character position in the original context. 
    offset_mapping = tokenized_examples.pop("offset_mapping")

    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # Label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Get the sequence corresponding to that example (to determine which is the context and the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, so this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [ ]:
# Apply function to all sentence pairs in dataset.
tokenized_datasets = dataset.map(prepare_train_features, batched=True, remove_columns=dataset["train"].column_names)

  0%|          | 0/88 [00:00<?, ?ba/s]

  0%|          | 0/11 [00:00<?, ?ba/s]

In [ ]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

# Instance model with an untrained QA head.
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Some weights of the model checkpoint at yiyanghkust/finbert-pretrain were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at yiyanghk

In [ ]:
batch_size = 32

In [ ]:
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"{model_name}-finetuned-squad",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
)

In [ ]:
from transformers import DefaultDataCollator

# Data collator to batch the processed examples together.
data_collator = DefaultDataCollator()

## Train

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 89204
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 8364


Epoch,Training Loss,Validation Loss
1,1.816900,1.709456
2,1.463900,1.597674
3,1.241600,1.605079


Saving model checkpoint to finbert-pretrain-finetuned-squad/checkpoint-500
Configuration saved in finbert-pretrain-finetuned-squad/checkpoint-500/config.json
Model weights saved in finbert-pretrain-finetuned-squad/checkpoint-500/pytorch_model.bin
tokenizer config file saved in finbert-pretrain-finetuned-squad/checkpoint-500/tokenizer_config.json
Special tokens file saved in finbert-pretrain-finetuned-squad/checkpoint-500/special_tokens_map.json
Saving model checkpoint to finbert-pretrain-finetuned-squad/checkpoint-1000
Configuration saved in finbert-pretrain-finetuned-squad/checkpoint-1000/config.json
Model weights saved in finbert-pretrain-finetuned-squad/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in finbert-pretrain-finetuned-squad/checkpoint-1000/tokenizer_config.json
Special tokens file saved in finbert-pretrain-finetuned-squad/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to finbert-pretrain-finetuned-squad/checkpoint-1500
Configuration saved i

TrainOutput(global_step=8364, training_loss=1.640006486335364, metrics={'train_runtime': 10230.0948, 'train_samples_per_second': 26.159, 'train_steps_per_second': 0.818, 'total_flos': 5.244461074772582e+16, 'train_loss': 1.640006486335364, 'epoch': 3.0})

In [ ]:
trainer.save_model(f"{model_name}-finetuned-squad/model")

Saving model checkpoint to finbert-pretrain-finetuned-squad/model
Configuration saved in finbert-pretrain-finetuned-squad/model/config.json
Model weights saved in finbert-pretrain-finetuned-squad/model/pytorch_model.bin
tokenizer config file saved in finbert-pretrain-finetuned-squad/model/tokenizer_config.json
Special tokens file saved in finbert-pretrain-finetuned-squad/model/special_tokens_map.json


# Save or Load Model

In [ ]:
from google.colab import files

files.download(f"{model_name}-finetuned-squad/model")

In [ ]:
!zip -r /content/model.zip /content/finbert-pretrain-finetuned-squad/model

  adding: content/finbert-pretrain-finetuned-squad/model/ (stored 0%)
  adding: content/finbert-pretrain-finetuned-squad/model/vocab.txt (deflated 50%)
  adding: content/finbert-pretrain-finetuned-squad/model/pytorch_model.bin (deflated 7%)
  adding: content/finbert-pretrain-finetuned-squad/model/special_tokens_map.json (deflated 42%)
  adding: content/finbert-pretrain-finetuned-squad/model/training_args.bin (deflated 48%)
  adding: content/finbert-pretrain-finetuned-squad/model/config.json (deflated 47%)
  adding: content/finbert-pretrain-finetuned-squad/model/tokenizer.json (deflated 70%)
  adding: content/finbert-pretrain-finetuned-squad/model/tokenizer_config.json (deflated 45%)


In [ ]:
files.download("model.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Testing

In [ ]:
!pip install transformers

In [ ]:
model_checkpoint = "content/finbert-pretrain-finetuned-squad/model/"

In [ ]:
from transformers import pipeline
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
from transformers import AutoTokenizer
import torch

In [ ]:
context = """
Machine learning (ML) is the study of computer algorithms that improve automatically through experience. It is seen as a part of artificial intelligence. Machine learning algorithms build a model based on sample data, known as "training data", in order to make predictions or decisions without being explicitly programmed to do so. Machine learning algorithms are used in a wide variety of applications, such as email filtering and computer vision, where it is difficult or unfeasible to develop conventional algorithms to perform the needed tasks.
"""
question = "what are the machine learning models based on?"

question, context = "Who was Jim Henson?", "Jim Henson was a nice puppet"

### QA-Pipeline

In [ ]:
question_answering = pipeline("question-answering", model=model_checkpoint, tokenizer=model_checkpoint)

In [ ]:
result = question_answering(question=question, context=context, device=0)
print("Answer:", result['answer'])
print("Score:", result['score'])

Answer: Machine learning
Score: 0.06451451778411865


### QA-Model, Tokenizer

In [ ]:
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
# inputs = tokenizer(question, text, return_tensors="pt")
inputs = tokenizer.encode_plus(question, context, add_special_tokens=True, return_tensors="pt")
input_ids = inputs["input_ids"].tolist()[0]

text_tokens = tokenizer.convert_ids_to_tokens(input_ids)
with torch.no_grad():
  outputs = model(**inputs)

answer_start = torch.argmax(outputs[0])  # Get the most likely beginning of answer with the argmax of the score
answer_end = torch.argmax(outputs[1]) + 1  # Get the most likely end of answer with the argmax of the score

answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))

print(f"Question: {question}")
print(f"Answer: {answer}\n")

Question: Who was Jim Henson?
Answer: a nice puppet

